In [1]:
!pip install git+https://github.com/huggingface/diffusers
!pip install -qq accelerate transformers ftfy

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-2w07b0xl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-2w07b0xl
  Resolved https://github.com/huggingface/diffusers to commit 74fd735eb073eb1d774b1ab4154a0876eb82f055
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
%pip install xformers
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!accelerate config default

Configuration already exists at /home/ubuntu/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [4]:
# MODEL_NAME="CompVis/stable-diffusion-v1-4"
MODEL_NAME="stabilityai/stable-diffusion-2-base"
INSTANCE_DIR="./vanellopenew"
OUTPUT_DIR="./dreambooth-out6/"
CLASS_DIR="./dreambooth-classimg3/"

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --train_text_encoder \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks girl" \
  --resolution=512 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --learning_rate=4e-7 \
  --lr_scheduler="constant" \
  --set_grads_to_none \
  --lr_warmup_steps=0 \
  --max_train_steps=520 \
  --enable_xformers_memory_efficient_attention \
  --use_8bit_adam \
  --class_data_dir=$CLASS_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --class_prompt="a photo of girl" \
  --num_class_images=800

/opt/conda/envs/pytorch/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/opt/conda/envs/pytorch/lib/python3.9/site-packages/accelerate/accelerator.py:258: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/accelerate/accelerator.py:375: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
06/07/2023 07:46:44 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate

In [13]:
from diffusers import DiffusionPipeline
import torch
torch.manual_seed(1286)

model_id = "./dreambooth-out4/"
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

pipe.safety_checker = None

prompt = "a photo of sks girl going shopping"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save("bed.png")

  0%|          | 0/50 [00:00<?, ?it/s]